In [7]:
import boto3
import json

In [3]:
iam_client = boto3.client('iam')

In [11]:
role_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [12]:
iam_client.create_role(
  RoleName='LambdaHFServerless',
  AssumeRolePolicyDocument=json.dumps(role_policy),
)

{'Role': {'Path': '/',
  'RoleName': 'LambdaHFServerless',
  'RoleId': 'AROA5F2FWVYHXEZSKWXAR',
  'Arn': 'arn:aws:iam::905847418383:role/LambdaHFServerless',
  'CreateDate': datetime.datetime(2021, 12, 23, 15, 57, 42, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Sid': '',
     'Effect': 'Allow',
     'Principal': {'Service': 'lambda.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '62de039f-8279-4d0e-9eb7-46fca0e6eda3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '62de039f-8279-4d0e-9eb7-46fca0e6eda3',
   'content-type': 'text/xml',
   'content-length': '817',
   'date': 'Thu, 23 Dec 2021 15:57:42 GMT'},
  'RetryAttempts': 0}}

In [41]:
iam_client.attach_role_policy(RoleName='LambdaHFServerless', PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

{'ResponseMetadata': {'RequestId': 'dafd522a-6a90-4a7a-8be2-8471b888e9a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dafd522a-6a90-4a7a-8be2-8471b888e9a7',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 23 Dec 2021 16:32:30 GMT'},
  'RetryAttempts': 0}}

In [42]:
endpoint_name = "hf-serverless-ep2021-12-23-10-42-53"

lambda_code = f"""
import os
import io
import boto3
import json
import csv


ENDPOINT_NAME = "{endpoint_name}"
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/json',
                                       Body=json.dumps(event, indent=4))
    result = json.loads(response['Body'].read().decode())
    pred = result[0]['label']

    return pred
"""

In [44]:
filename = "lambda"

In [45]:
with open(f"{filename}.py", "w") as f:
    f.write(lambda_code)

In [46]:
from zipfile import ZipFile

In [47]:
with ZipFile('lambda.zip', 'w') as myzip:
    myzip.write('lambda.py')

In [48]:
lambda_client = boto3.client('lambda')

with open('lambda.zip', 'rb') as f:
    zipped_code = f.read()
  
role = iam_client.get_role(RoleName='LambdaHFServerless')

response = lambda_client.create_function(
    FunctionName='HFServerlessLambda',
    Runtime='python3.9',
    Role=role['Role']['Arn'],
    Handler=f'{filename}.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300,
)

In [49]:
payload = {
  "inputs": "This is a great test"
}

In [ ]:
response = lambda_client.invoke(FunctionName   = "HFServerlessLambda",
                InvocationType = "RequestResponse",
                Payload        = bytes(json.dumps(payload), encoding='utf8'))

In [55]:
json.loads(response['Payload'].read())

'LABEL_1'